In [1]:
import allel
import numpy as np
import pandas as pd

### Run G123 on Darlingi

In [3]:
ls

Adarlingi_annotation_example.vcf            ag-vampir-002.annot.vcf
Adarlingi_example_selection_region.vcf      darlingi-diplotype-clustering.ipynb
Adarlingi_example_selection_region.vcf.zip  darlingi-g123.ipynb
Adarlingi_example.vcf                       test.csv


In [10]:
vcf = allel.read_vcf("Adarlingi_annotation_example.vcf", fields="*")

In [14]:
import pandas as pd
import numpy as np

def parse_snpeff_annotations(annotations: np.ndarray) -> pd.DataFrame:
    """
    Parse SnpEff annotations from a numpy array of strings into a pandas DataFrame.
    
    Args:
        annotations (np.ndarray): Array of SnpEff annotation strings
        
    Returns:
        pd.DataFrame: DataFrame with parsed annotation fields
    """
    # Define column names based on SnpEff annotation format
    columns = [
        'allele', 'effect', 'impact', 'gene_name', 'gene_id',
        'feature_type', 'feature_id', 'transcript_biotype', 'rank_total',
        'hgvs_c', 'hgvs_p', 'cdna_pos', 'cds_pos', 'protein_pos',
        'distance', 'errors'
    ]
    
    # Split annotations and create list of lists
    parsed_data = [ann.split('|') for ann in annotations]
    
    # Create DataFrame
    df = pd.DataFrame(parsed_data, columns=columns)
    
    return df

In [20]:
vcf = allel.read_vcf("Adarlingi_example_selection_region.vcf", fields="*")
vcf.keys()

dict_keys(['samples', 'calldata/AD', 'calldata/DP', 'calldata/GQ', 'calldata/GT', 'calldata/MIN_DP', 'calldata/PGT', 'calldata/PID', 'calldata/PL', 'calldata/PS', 'calldata/RGQ', 'calldata/SB', 'variants/AC', 'variants/AF', 'variants/ALT', 'variants/AN', 'variants/AS_BaseQRankSum', 'variants/AS_FS', 'variants/AS_InbreedingCoeff', 'variants/AS_MQ', 'variants/AS_MQRankSum', 'variants/AS_QD', 'variants/AS_QUALapprox', 'variants/AS_ReadPosRankSum', 'variants/AS_SOR', 'variants/AS_VarDP', 'variants/BaseQRankSum', 'variants/CHROM', 'variants/DP', 'variants/END', 'variants/ExcessHet', 'variants/FILTER_LowQual', 'variants/FILTER_PASS', 'variants/FS', 'variants/HAPCOMP', 'variants/HAPDOM', 'variants/HEC', 'variants/ID', 'variants/InbreedingCoeff', 'variants/MLEAC', 'variants/MLEAF', 'variants/MQ', 'variants/MQRankSum', 'variants/MQ_DP', 'variants/NDA', 'variants/POS', 'variants/QD', 'variants/QUAL', 'variants/QUALapprox', 'variants/RAW_GT_COUNT', 'variants/RAW_MQandDP', 'variants/REF', 'variant

In [21]:
df_ann = parse_snpeff_annotations(vcf['variants/ANN'])
df_ann

KeyError: 'variants/ANN'

In [3]:
geno = allel.GenotypeArray(vcf['calldata/GT'])
pos = vcf['variants/POS']
samples = vcf['samples']

In [4]:
df_samples = pd.DataFrame({'sampleID':samples})['sampleID'].str.split("_", expand=True).iloc[:, :2]
df_samples.columns = ['loc1', 'loc2']